In [1]:
import pandas as pd
import sqlalchemy as sa
from sqlalchemy import Table, MetaData, Column, Integer, create_engine
import urllib
import pyodbc 

### Build and Establish Connection to DB

In [18]:
#server = ''
#database = ''
#username = ''
#password = ''

params = urllib.parse.quote_plus('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)

engine = sa.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

In [19]:
connection = engine.connect()

### Read DB Contents

In [27]:
query = """
select schema_name(t.schema_id) as schema_name,
       t.name as table_name,
       t.create_date,
       t.modify_date
from sys.tables t
order by schema_name,
         table_name;
"""

rows = pd.read_sql_query(query, connection)
rows

,schema_name,table_name,create_date,modify_date
0,dbo,BuildVersion,2022-05-15 20:04:00.113,2022-05-15 20:04:01.767
1,dbo,ErrorLog,2022-05-15 20:04:00.113,2022-05-15 20:04:01.767
2,dbo,Rel_Building,2022-05-17 22:49:05.513,2022-05-19 22:05:23.507
3,dbo,Rel_BuyoutAgreements,2022-05-17 22:47:28.627,2022-05-19 22:05:23.503
4,dbo,Rel_Demographics,2022-05-17 22:34:37.987,2022-05-19 22:21:30.853
5,dbo,Rel_Eviction_Notices,2022-05-17 21:59:47.223,2022-05-17 21:59:47.223
6,dbo,Rel_Location,2022-05-17 22:25:35.210,2022-05-19 22:21:30.853
7,dbo,SF_Addresses_With_Units_Raw,2022-05-19 01:46:22.717,2022-05-19 01:46:22.793
8,dbo,SF_Buyout_Agreements_Raw,2022-05-16 00:27:00.917,2022-05-16 00:27:00.943
9,dbo,SF_Eviction_Notices_Raw,2022-05-16 00:41:20.143,2022-05-16 00:41:20.190


### Clean Up Sample Schema from DB

In [22]:
#find foreign keys preventing tables from being deleted
statement = """
EXEC sp_fkeys @pktable_name = 'Address',@pktable_owner    = N'SalesLT';
"""

response = pd.read_sql_query(statement, connection)
response

,PKTABLE_QUALIFIER,PKTABLE_OWNER,PKTABLE_NAME,PKCOLUMN_NAME,FKTABLE_QUALIFIER,FKTABLE_OWNER,FKTABLE_NAME,FKCOLUMN_NAME,KEY_SEQ,UPDATE_RULE,DELETE_RULE,FK_NAME,PK_NAME,DEFERRABILITY


In [ ]:
#find dependencies preventing tables from being deleted
statement = """
EXEC sp_depends 'SalesLT.ProductModelProductDescription';
"""

response = pd.read_sql_query(statement, connection)
response

In [24]:
#find remaining schema objects
statement = """
SELECT *
FROM sys.objects WHERE schema_id = SCHEMA_ID('SalesLT');
"""

response = pd.read_sql_query(statement, connection)
response

,name,object_id,principal_id,schema_id,parent_object_id,type,type_desc,create_date,modify_date,is_ms_shipped,is_published,is_schema_published


In [ ]:
#drop schema objects one at a time, then drop schema
statement = """
DROP SCHEMA SalesLT;
"""

connection.execute(statement)

In [ ]:
#drop schema objects one at a time, then drop schema
statement = """
DROP TABLE [dbo].[Rel_Eviction_Notices];
"""

connection.execute(statement)

### Create Normalized Tables

In [ ]:
#create eviction notices table
statement = """
CREATE TABLE Rel_Eviction_Notices
(
    eviction_auto_id                                     uniqueidentifier PRIMARY KEY NOT NULL DEFAULT newid(),
    eviction_id                                          VARCHAR(MAX) NOT NULL,
    file_date                                            DATE NOT NULL,
    non_payment                                          BIT NOT NULL,
    breach                                               BIT NOT NULL,
    nuisance                                             BIT NOT NULL,
    illegal_use                                          BIT NOT NULL,
    failure_to_sign_renewal                              BIT NOT NULL,
    access_denial                                        BIT NOT NULL,
    unapproved_subtenant                                 BIT NOT NULL,
    owner_move_in                                        BIT NOT NULL,
    demolition                                           BIT NOT NULL,
    capital_improvement                                  BIT NOT NULL,
    substantial_rehab                                    BIT NOT NULL,
    ellis_act_withdrawal                                 BIT NOT NULL,
    condo_conversion                                     BIT NOT NULL,
    roommate_same_unit                                   BIT NOT NULL,
    other_cause                                          BIT NOT NULL,
    late_payments                                        BIT NOT NULL,
    lead_remediation                                     BIT NOT NULL,
    development                                          BIT NOT NULL,
    good_samaritan_ends                                  BIT NOT NULL,
    constraints_date                                     DATE
)
"""

connection.execute(statement)

In [ ]:
#create location table
statement = """
CREATE TABLE Rel_Location
(
    location_id                                          uniqueidentifier PRIMARY KEY NOT NULL DEFAULT newid(),
    block_level_address                                  VARCHAR NOT NULL,
    zipcode                                              INT NOT NULL,
    current_supervisor_districts                         TINYINT,
    analysis_neighborhoods                               TINYINT,
    city                                                 VARCHAR NOT NULL,
    state                                                VARCHAR NOT NULL,
    supervisor_district                                  TINYINT NOT NULL,
    neighborhoods_analysis_boundaries                    VARCHAR,
    location                                             GEOGRAPHY,
    shape                                                GEOGRAPHY,
    sf_find_neighborhoods                                TINYINT,
    current_police_districts                             TINYINT,
    cbd_bid_gbd_boundaries                               TINYINT,
    central_market_tenderloin_boundary                   TINYINT,
    areas_of_vulnerability                               TINYINT,
    central_market_tenderloin_boundary_polygon           TINYINT,
    fix_it_zones                                         TINYINT,
    neighborhoods                                        TINYINT
)
"""

connection.execute(statement)

In [ ]:
#create demographics table
statement = """
CREATE TABLE Rel_Demographics
(
    demographic_id                                          uniqueidentifier PRIMARY KEY NOT NULL DEFAULT newid(),
    median_household_income                                 INT NOT NULL,
    population                                              INT NOT NULL,
    zipcode                                                 INT NOT NULL,
    avg_population_age                                      FLOAT NOT NULL,
    people_per_sq_mile                                      FLOAT NOT NULL
)
"""

connection.execute(statement)

In [ ]:
#create buyout agreements table
statement = """
CREATE TABLE Rel_BuyoutAgreements
(
    case_number_auto_id                                          uniqueidentifier PRIMARY KEY NOT NULL DEFAULT newid(),
    case_number                                                  VARCHAR NOT NULL,
    location_id                                                  uniqueidentifier NOT NULL FOREIGN KEY REFERENCES Rel_Location(location_id),
    pre_buyout_disclosure_declaration                            DATE,
    buyout_agreement_date                                        DATE,
    address                                                      VARCHAR NOT NULL,
    buyout_amount                                                INT,
    unknown_amount                                               BIT,
    other_consideration                                          BIT
)
"""

connection.execute(statement)

In [ ]:
#create building table
statement = """
CREATE TABLE Rel_Building
(
    building_id                                          uniqueidentifier PRIMARY KEY NOT NULL DEFAULT newid(),
    number_of_tenants                                    TINYINT
)
"""

connection.execute(statement)

### Add Missing Foreign Keys

In [ ]:
#add building_id column to buyout agreements
statement = """
ALTER TABLE dbo.Rel_BuyoutAgreements
ADD building_id uniqueidentifier;
"""

connection.execute(statement)

In [ ]:
#make building_id FK in buyout agreements
statement = """
ALTER TABLE dbo.Rel_BuyoutAgreements
ADD CONSTRAINT building_id
FOREIGN KEY (building_id) REFERENCES dbo.Rel_Building(building_id);
"""

connection.execute(statement)

In [ ]:
#add demographic_id column to location
statement = """
ALTER TABLE dbo.Rel_Location
ADD demographic uniqueidentifier;
"""

connection.execute(statement)

In [ ]:
#make demographic_id FK in location
statement = """
ALTER TABLE dbo.Rel_Location
ADD CONSTRAINT demographic_id
FOREIGN KEY (demographic_id) REFERENCES dbo.Rel_Demographics(demographic_id);
"""

connection.execute(statement)

In [30]:
#add location_id to eviction notices
statement = """
ALTER TABLE dbo.Rel_Eviction_Notices
ADD location_id uniqueidentifier;
"""

connection.execute(statement)

In [31]:
#make location_id FK in eviction notices
statement = """
ALTER TABLE dbo.Rel_Eviction_Notices
ADD CONSTRAINT location_id
FOREIGN KEY (location_id) REFERENCES dbo.Rel_Location(location_id);
"""

connection.execute(statement)